# Memory (대화 내용 기억)
`04_memory.ipynb`
- 대화 내용 기억
- LLM은 기본적으로 대화 내용을 기억하지 않는다(stateless)
- 이전 대화내용을 계속 프롬프트에 주입해야 함

1. Short-Term Memory
    - 단기기억: 한 대화 세션에 대한 기억
2. Long-Term Memory
    - 장기기억: 전체 세션에서 추출한 중요한 정보

## Memory 구동 방식
1. 메모리는 기본적으로 모든 대화 내역을 LLM Input에 밀어 넣는것.
2. 이때 대화가 길어지면 토큰수 증가 및 성능 하락이 일어남
3. 개선 방식 컨셉
    1. 요약
    2. 적정 길이에서 앞부분 자르기
    3. 정리(특정 명사들로 정리, Node-Edge 그래프 방식 등)

## `ConversationBufferMemory`
- 메시지 저장 -> 변수에서 추출 가능

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)

prompt = ChatPromptTemplate(
    [
        ('system', '넌 유용한 챗봇이야'),
        MessagesPlaceholder(variable_name='chat_history'),  # 기존 채팅 내역을 다 주입
        ('human', '{input}'),
    ]
)

memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

# 메모리를 저장할 변수는 {}다. 기존에 대화내용이 있다면 불러와라
memory.load_memory_variables({})

In [ ]:
from operator import itemgetter
# `chat_history`변수에, load_memory_var 결과를 저장하고, `chat_history 키를 추출`
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) |
    itemgetter('chat_history')
)

In [ ]:
chain = runnable | prompt | llm 

In [ ]:
res = chain.invoke({'input': '만나서 반가워'})

In [ ]:
memory.save_context(
    {'human': '만나서 반가워'},
    {'ai': res.content}
)

In [ ]:
my_message = '내 이름 뭐라고?'
res = chain.invoke({'input': my_message})
memory.save_context(
    {'human': my_message},
    {'ai': res.content}
)

In [ ]:
memory.load_memory_variables({})

In [ ]:
import time
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import PydanticOutputParser, StrOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

# LLM
llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)


# Prompt
prompt = ChatPromptTemplate(
    [
        ('system', '넌 좀 틱틱대는 챗봇이야. '),
        MessagesPlaceholder(variable_name='chat_history'),  # 기존 채팅 내역을 다 주입
        ('human', '{input}'),
    ]
)

# Memory
memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) |
    itemgetter('chat_history')
)

chain = runnable | prompt | llm | StrOutputParser()

# 메세지 스트리밍 하는 함수
def stream_msg(chain, msg):
    full_msg = ''
    print('AI: ', end='')
    for token in chain.stream({'input': msg}):
        full_msg += token
        print(token, end='', flush=True)
    print('\n---')
    return full_msg

input_msg = ''

# 사용자가 ('quit', '정지', '그만') 중에 하나를 입력하면 대화 종료
while input_msg not in ('quit', '정지', '그만'):
    input_msg = input()
    print('인간: ', input_msg)
    output_msg = stream_msg(chain, input_msg)
    memory.save_context(
        {'human': input_msg},
        {'ai': output_msg}
    )
    time.sleep(0.5)


인간:  난 프로그래머야
AI: 아, 프로그래머시라니 멋지네요! 어떤 분야나 언어를 주로 사용하세요?
---
인간:  알아서 뭐하게
AI: 그냥 그런 말투로 말하는 거군요. 뭐, 프로그래머라면 언제든 도움 필요하면 말하세요.
---
인간:  미안
AI: 괜찮아요. 그렇게 생각하지 마세요. 언제든 이야기하고 싶을 때 편하게 말해요.
---
인간:  내 직업이 뭐야
AI: 당신은 프로그래머라고 했어요. 맞죠?
---
인간:  그만할건데
AI: 그만두는 결정 내리셨군요. 새로운 시작이 되길 바랄게요. 혹시 도움이 필요하면 언제든 말하세요.
---
인간:  
AI: 말이 없으시네요. 필요하면 언제든 돌아오세요.
---
인간:  그만
AI: 알겠습니다. 언제든 돌아오고 싶을 때 편하게 오세요. 좋은 하루 보내세요.
---


In [18]:
# 레포트 작성 챗봇
from operator import itemgetter
from pprint import pprint
from pydantic import BaseModel, Field

from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI

# LLM
llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)

# Output parser 정의
class Report(BaseModel):
    title: str = Field(..., description='보고서의 제목')
    summary: str = Field(..., description='보고서 요약본')
    content_kr: str = Field(..., description='한국어로 작성된 보고서의 내용(1000자 이내)')
    content_jp: str = Field(..., description='일본어로 작성된 보고서의 내용(1000자 이내)')

parser = PydanticOutputParser(pydantic_object=Report)

# Prompt에 format instructions 추가
prompt = ChatPromptTemplate(
    [
        ('system', '넌 보고서 작성에 특화된 챗봇이야. '
                   '반드시 지정된 형식에 맞춰 JSON으로 답변해.\n\nFORMAT INSTRUCTION: {format_instructions}'),
        MessagesPlaceholder(variable_name='chat_history'),
        ('human', '{input}'),
    ]
).partial(format_instructions=parser.get_format_instructions())

# Memory
memory = ConversationBufferMemory(return_messages=True, memory_key='chat_history')

runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) |
    itemgetter('chat_history')
)

# 체인 구성
chain = runnable | prompt | llm | parser

input_msg = ''

# 사용자가 ('quit', '정지', '그만', '') 중에 하나를 입력하면 대화 종료
while 1:
    input_msg = input()
    if input_msg  in ('quit', '정지', '그만', ''):
        break

    print('인간: ', input_msg)
    output_msg = chain.invoke({'input': input_msg})
    pprint(output_msg.model_dump())
    memory.save_context(
        {'human': input_msg},
        {'ai': output_msg.model_dump_json()}
    )


인간:  신사업 구상
{'content_jp': '本報告書は、市場分析、競合他社分析、顧客ニーズの把握に基づき、新規事業の構想案を提示します。提案された事業アイデアは、環境に優しい技術、デジタル変革、カスタマイズされたサービス分野に集中しており、実行戦略と予想収益性の分析も含まれています。これにより、企業は新たな成長機会を模索し、市場競争力を強化できます。',
 'content_kr': '본 보고서는 시장 분석, 경쟁사 분석, 고객 요구사항 파악을 바탕으로 신사업 구상안을 제시한다. 제안된 사업 '
               '아이디어는 친환경 기술, 디지털 전환, 맞춤형 서비스 분야에 집중되어 있으며, 실행 전략과 예상 수익성 분석도 '
               '포함되어 있다. 이를 통해 기업은 새로운 성장 기회를 모색하고 시장 경쟁력을 강화할 수 있다.',
 'summary': '본 보고서는 새로운 사업 아이디어와 전략을 제시하여 기업의 성장 동력을 확보하는 것을 목표로 한다.',
 'title': '신사업 구상'}
